In [2]:
import scipy.special as ssp
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as ss
from pylab import *
import random as rnd
from numba import jit
from statistics import NormalDist

from statsmodels.tsa.arima_process import ArmaProcess

In [2]:
@jit(nopython=True)
def zt(p, zt, n):
    return np.array(
    [0 if i > 2 * p else (float(zt) if i < p else float(-zt)) for i in [rnd.random() for i in range(n)]])

In [3]:
@jit(nopython=True)
def autocov(X, lag):
    return np.array([1 / len(X) * (np.sum((X[:len(X) - i] - np.mean(X)) * (X[i:] - np.mean(X)))) for i in range(lag[0], lag[1] + 1)])

In [4]:
@jit(nopython=True)
def autocorr(X, lag):
    return np.array([i / autocov(X, [0, 1])[0] for i in autocov(X, lag)])

In [5]:
@jit(nopython=True)
def binom(n, k):
    k = k if k > n/2 else n-k
    n = np.arange(k + 1, n + 1, 1)
    k = np.arange(1, n[-1] - k + 1, 1)
    return np.prod(n)/np.prod(k)

In [6]:
binom(1000, 2)

499500.0

In [7]:
@jit(nopython=True)
def Q(X, k):
    values = np.array([abs(X[j] - X[i]) for i in range(len(X)) for j in range(i)])
    values = np.array(sorted(values))
    return k * values[int(np.floor((binom(len(X), 2) + 2) / 4))]

In [8]:
Q(np.array([rnd.random() for i in range(100)]), 2.11)

0.28628568053052755

In [9]:
@jit(nopython=True)
def autocov1(X, lag, k):
    acvf = []
    for h in range(lag[0], lag[1] + 1):
        if h == 0:
            u = X
        else:
            u = X[:-h]
        v = X[h:]
        acvf.append(1 / 4 * ((Q(u + v, k)) ** 2 - (Q(u - v, k)) ** 2))
    return np.array(acvf)

In [10]:
autocov1(np.array([rnd.random() for i in range(100)]), np.array([0, 5]),2.11)

array([ 0.0811557 ,  0.01573544,  0.00197485, -0.01572118, -0.01436024,
        0.00741606])

In [11]:
@jit(nopython=True)
def autocorr1(X, lag, k):
    acf = []
    for h in range(lag[0], lag[1] + 1):
        if h == 0:
            u = X
        else:
            u = X[:-h]
        v = X[h:]
        acf.append(((Q(u + v, k)) ** 2 - (Q(u - v, k)) ** 2) / ((Q(u + v, k)) ** 2 + (Q(u - v, k)) ** 2))
    return acf

In [12]:
autocorr1(np.array([rnd.random() for i in range(100)]), np.array([0, 5]),2.11)

[1.0,
 -0.21570375292232433,
 0.03544259167915104,
 -0.09201494788608806,
 -0.016209361293102555,
 -0.03357526737292587]

In [13]:
@jit(nopython=True)
def g(alfa, n):
    return int(floor(n * alfa))

In [14]:
@jit(nopython=True)
def L(X, t, alfa):
    Y = np.array(sorted(X))
    n = len(X)
    _g = g(alfa, n)
    if Y[_g] < X[t] and X[t] < Y[n - _g]:
        return 1
    return 0

In [15]:
L(np.array([rnd.random() for i in range(100)]), 5,0.015)

1

In [16]:
@jit(nopython=True)
def mean_X(X, alfa):
    suma1 = np.sum(np.array([L(X, i, alfa) for i in range(len(X))]))
    suma2 = np.sum(np.array([X[i] * L(X, i, alfa) for i in range(len(X))]))
    return 1 / suma1 * suma2

In [17]:
mean_X(np.array([rnd.random() for i in range(100)]),0.015)

0.49406003632961054

In [18]:
@jit(nopython=True)
def autocov2(X, lag, alfa):
    n = len(X)
    L_vector = np.array([L(X, i, alfa) for i in range(n)])
    acvf = []
    for h in range(lag[0], lag[1] + 1):
        if h == 0:
            suma1 = np.sum(L_vector)
            suma2 = np.sum(X ** 2 * L_vector)
        else:
            suma1 = np.sum(L_vector[:-h] * L_vector[h:])
            suma2 = np.sum(X[:-h] - mean_X(X, alfa) * X[h:] - mean_X(X, alfa) *
                L_vector[:-h] * L_vector[h:])
        acvf.append(suma2 / suma1)
    return np.array(acvf)

In [19]:
autocov2(np.array([rnd.random() for i in range(100)]), np.array([0,5]),0.015)

array([ 0.28725466, -0.19348722, -0.19228444, -0.19048914, -0.19640445,
       -0.20092154])

In [20]:
@jit(nopython=True)
def autocorr2(X, lag, alfa):
    acvf = autocov2(X, lag, alfa)[0]
    return np.array([i / acvf for i in autocov2(X, lag, alfa)])

In [21]:
autocorr2(np.array([rnd.random() for i in range(100)]), np.array([0,5]),0.015)

array([ 1.        , -0.81017367, -0.82226007, -0.82359716, -0.82863895,
       -0.82097026])

In [22]:
@jit(nopython=True)
def autocorr4(X, lag):
    X = X - np.mean(X)
    acf = [1.0]
    denominator = np.median(np.array([i**2 for i in X]))
    for h in range(lag[0] + 1, lag[1] + 1):
        acf.append(np.median(X[:-h] * X[h:])/denominator)
    return acf

In [23]:
autocorr4(np.array([rnd.random() for i in range(100)]), np.array([0,5]))

[1.0,
 0.0891280340062436,
 0.0032869993003648576,
 -0.016158662483720145,
 -0.011590271498981494,
 0.038491195065703455]

In [30]:
@jit
def J(x):
    return float(NormalDist().inv_cdf(x))

In [31]:
J(0.3)

C:\Users\piter\AppData\Local\Temp/ipykernel_33476/1169055119.py:1: NumbaWarning: 
Compilation is falling back to object mode WITH looplifting enabled because Function "J" failed type inference due to: Untyped global name 'NormalDist': Cannot determine Numba type of <class 'type'>

File "..\..\..\..\..\AppData\Local\Temp\ipykernel_33476\1169055119.py", line 3:
<source missing, REPL/exec in use?>

  @jit
C:\Users\piter\anaconda3\lib\site-packages\numba\core\object_mode_passes.py:151: NumbaWarning: Function "J" was compiled in object mode without forceobj=True.

File "..\..\..\..\..\AppData\Local\Temp\ipykernel_33476\1169055119.py", line 1:
<source missing, REPL/exec in use?>

  warnings.warn(errors.NumbaWarning(warn_msg,
C:\Users\piter\anaconda3\lib\site-packages\numba\core\object_mode_passes.py:161: NumbaDeprecationWarning: 
Fall-back from the nopython compilation path to the object mode compilation path has been detected, this is deprecated behaviour.

For more information visit https:

-0.5244005127080407

In [32]:
@jit
def c(X):
    n = len(X)
    R = np.array(sorted(X))
    return 1 / np.sum([(J((list(X).index(R[i]) + 1) / (n + 1)))**2 for i in range(n)])

In [33]:
c(np.array([rnd.random() for i in range(100)]))

C:\Users\piter\AppData\Local\Temp/ipykernel_33476/2760330457.py:1: NumbaWarning: 
Compilation is falling back to object mode WITH looplifting enabled because Function "c" failed type inference due to: Invalid use of type(CPUDispatcher(<function J at 0x000001FD0AF6BE50>)) with parameters (float64)

During: resolving callee type: type(CPUDispatcher(<function J at 0x000001FD0AF6BE50>))
During: typing of call at C:\Users\piter\AppData\Local\Temp/ipykernel_33476/2760330457.py (5)


File "..\..\..\..\..\AppData\Local\Temp\ipykernel_33476\2760330457.py", line 5:
<source missing, REPL/exec in use?>

  @jit
C:\Users\piter\anaconda3\lib\site-packages\numba\core\object_mode_passes.py:151: NumbaWarning: Function "c" was compiled in object mode without forceobj=True.

File "..\..\..\..\..\AppData\Local\Temp\ipykernel_33476\2760330457.py", line 1:
<source missing, REPL/exec in use?>

  warnings.warn(errors.NumbaWarning(warn_msg,
C:\Users\piter\anaconda3\lib\site-packages\numba\core\object_mode_passe

0.010837050605284464

In [48]:
@jit
def autocorr3(X, lag):
    acf = []
    n = len(X)
    R = np.array(sorted(X))
    for h in range(lag[0], lag[1] + 1):
        suma = 0
        for i in range(n-h):
            a = list(X).index(R[i])
            b = list(X).index(R[i + h])
            suma += J((np.where(R[i] == X)[0] + 1)/ (n+1)) * J((np.where(R[i+h] == X)[0] + 1)/ (n+1))
        acf.append(c(X) * suma)
    return acf

In [49]:
autocorr3(np.array([rnd.random() for i in range(100)]), np.array([0,5]))

C:\Users\piter\AppData\Local\Temp/ipykernel_33476/1458876154.py:1: NumbaWarning: 
Compilation is falling back to object mode WITH looplifting enabled because Function "autocorr3" failed type inference due to: Invalid use of type(CPUDispatcher(<function J at 0x000001FD0AF6BE50>)) with parameters (array(float64, 1d, C))

During: resolving callee type: type(CPUDispatcher(<function J at 0x000001FD0AF6BE50>))
During: typing of call at C:\Users\piter\AppData\Local\Temp/ipykernel_33476/1458876154.py (11)


File "..\..\..\..\..\AppData\Local\Temp\ipykernel_33476\1458876154.py", line 11:
<source missing, REPL/exec in use?>

  @jit
C:\Users\piter\AppData\Local\Temp/ipykernel_33476/1458876154.py:1: NumbaWarning: 
Compilation is falling back to object mode WITHOUT looplifting enabled because Function "autocorr3" failed type inference due to: Cannot determine Numba type of <class 'numba.core.dispatcher.LiftedLoop'>

File "..\..\..\..\..\AppData\Local\Temp\ipykernel_33476\1458876154.py", line 6:
<so

[1.0000000000000002,
 0.005913328131875116,
 0.06324550082255595,
 0.007274151866831367,
 -0.12123603380012053,
 -0.15280749301462884]

In [65]:
@jit(nopython=True)
def autocorr5(X, lag):
    n = len(X)
    return [1/((n - h)*(n - h - 1)) * np.sum(np.array([np.sign((X[i] - X[j]) * (X[i + h] - X[j + h]))
                for i in range(n - h) for j in range(i)])) for h in range(lag[0], lag[1] + 1)]

In [66]:
autocorr5(np.array([rnd.random() for i in range(100)]), np.array([0,5]))

[0.5,
 -0.03906411049268192,
 0.046812539448769196,
 -0.007302405498281787,
 -0.01118421052631579,
 -0.07625979843225085]

In [75]:
@jit(nopython=True)
def autocorr6(X, lag):
    n = len(X)
    mediana = np.median(X)
    acf = [1/(n - h) * np.sum(np.sign((X[:-h] - mediana) *
    (X[h:] - mediana))) for h in range(lag[0] + 1, lag[1] + 1)]
    return [1, *acf]

In [76]:
autocorr6(np.array([rnd.random() for i in range(100)]), np.array([0,5]))

[1.0,
 -0.07070707070707072,
 -0.2040816326530612,
 -0.09278350515463918,
 -0.0625,
 0.11578947368421053]

In [101]:
def zmatrix(X, k):
    return np.matrix([[*[0 for i in range(j)], *[X[i] for i in range(k)], *[0 for i in range(k - j)]] for j in range(k+1)])

In [103]:
def gamma_k(X, k):
    z_matr = zmatrix(X, k)
    return (z_matr * z_matr.transpose())/len(X)

In [104]:
def Xi(X, k, i, j):
    gamm = gamma_k(X, k)
    return gamm[i - 1, j - 1]/np.sqrt(gamm[i - 1, i - 1] * gamm[j - 1, j - 1])

In [105]:
def autocorr7(X, lag):
    k = lag[1]
    return [1/(k - h + 1)*sum([Xi(X, k, i, i + h) for i in range(1, k - h + 2)]) for h in range(lag[0], lag[1] + 1)]

In [106]:
@jit(nopython=True)
def autocorr8(X, lag):
    return [(np.var(X+X) - np.var(X - X))/(np.var(X+X) + np.var(X - X)) if h == 0 else
            (np.var(X[h:]+X[:-h]) - np.var(X[h:] - X[:-h]))/(np.var(X[h:]+X[:-h]) + np.var(X[h:] - X[:-h]))
            for h in range(lag[0], lag[1] + 1)]

In [107]:
autocorr8(np.array([rnd.random() for i in range(100)]), np.array([0,5]))

[1.0,
 -0.059842716495883935,
 -0.10899296499540402,
 0.18428145582216268,
 0.030702153939854142,
 0.0402330277350975]